**Getting Data**

In previous notebook we downloaded photos and save them on google drive 

In [1]:
"""libraries we use"""
!sudo apt-get install unzip
from google.colab import drive
from google.colab import files
import csv 
drive.mount('/content/drive')
import os.path

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Upload csv file to read data from it

In [2]:
!cp drive/My\ Drive/Memotion/data_train.csv /content/
!cp drive/My\ Drive/Memotion/data_test.csv /content/


train_filename = 'data_train.csv'
test_filename = 'data_test.csv'

# reading the file into a list
train_data = [] 
test_data = []
with open(train_filename, 'r') as csvfile: 
    csvreader = csv.reader(csvfile)  
    for row in csvreader: 
        train_data.append(row) 
csvfile.close()

with open(test_filename, 'r') as csvfile: 
    csvreader = csv.reader(csvfile)  
    for row in csvreader: 
        test_data.append(row) 
csvfile.close()

print('Example :')
print(train_data[0])
print(train_data[1])

Example :
['Image_name', 'Image_URL', 'OCR_extracted_text', 'corrected_text', 'Humour', 'Sarcasm', 'offensive', 'Motivational', 'Overall_Sentiment']
['10_year_2r94rv.jpg', 'https://i.imgflip.com/2r94rv.jpg', 'LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIKUT TREND PLAY THE 10 YEARS CHALLENGE AT FACEBOOK imgflip.com ', 'LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIKUT TREND PLAY THE 10 YEARS CHALLENGE AT FACEBOOK imgflip.com ', 'hilarious', 'general', 'not_offensive', 'not_motivational', 'very_positive']


Some lines has incorrect data e.g. in some lines corrected ocr is empty , in addition in some lines url contains wrong data so we delet lines which does not have 9 columns

In [3]:
del train_data[0] # header part
counter = 0  
for line in train_data[:] :
  if(line[-1] == ''):
    train_data.remove(line)
print('new train data has' ,str(len(train_data)) , 'lines!')

del test_data[0] # header part
counter = 0  
for line in test_data[:] :
  if(line[-1] == ''):
    test_data.remove(line)
print('new test data has' ,str(len(test_data)) , 'lines!')

new train data has 6521 lines!
new test data has 914 lines!


Some of corrected OCR still contains websites url or username ,We try to delete most of them

In [0]:
for line in train_data :
  corrected_ocr = ' '.join(item for item in line[3].split() if not (item.startswith('@') or item.endswith('.com') or item.endswith('.net')))
  line[3] = corrected_ocr

for line in test_data :
  corrected_ocr = ' '.join(item for item in line[3].split() if not (item.startswith('@') or item.endswith('.com') or item.endswith('.net')))
  line[3] = corrected_ocr

get prepared dataset from google drive

In [5]:
!cp drive/My\ Drive/Memotion/train_dataset.zip /content/
!cp drive/My\ Drive/Memotion/test_dataset.zip /content/

!unzip train_dataset.zip
!unzip test_dataset.zip

Archive:  train_dataset.zip
   creating: train_dataset/
  inflating: train_dataset/bean_rmx-rmx-just-mr-bean_o_712217.jpg  
  inflating: train_dataset/bean_1yuex6.jpg  
  inflating: train_dataset/barney_how-i-met-your-mother-season-67_o_261072.jpg  
  inflating: train_dataset/bean_baby-mr-bean-funny-picture-3924201.png  
  inflating: train_dataset/sports_Sports-Memes-35.jpg  
  inflating: train_dataset/zeck_need-a-meme-idea-why-not-zuckerberg_o_975261.jpg  
  inflating: train_dataset/seal_f12019a559944c15ce43deb0e8c51732511a5198d87080311eb98b519f00c113.jpg  
  inflating: train_dataset/got_7-5cb5c9acce0f6__700.jpg  
  inflating: train_dataset/hillary_feminists-will-vote-for-hillary-purely-because-she-amp-039-s-a-woman_o_5033165.jpg  
  inflating: train_dataset/liam_e70c36bd22efd92cc7ff1919f3c8a3809e567d0fc78202ff53f21d2cf0f6c6cc.jpg  
  inflating: train_dataset/harry_Harry-Potter-21.jpg  
  inflating: train_dataset/chuck_e11094bc6bda894c.jpg  
  inflating: train_dataset/decaprio_leonard

some photos can not convert to jpeg so we delete them

In [6]:
import os
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

TRAIN_DATASET = 'train_dataset/'
TEST_DATASET = 'test_dataset/'

for image in os.listdir(TRAIN_DATASET) :
  try :
    img = tf.io.read_file(TRAIN_DATASET + image)
    img = tf.image.decode_jpeg(img,channels=3)
  except:
    os.remove(TRAIN_DATASET + image)
print("number of photos in train : " , len(os.listdir(TRAIN_DATASET)))

for image in os.listdir(TEST_DATASET) :
  try :
    img = tf.io.read_file(TEST_DATASET + image)
    img = tf.image.decode_jpeg(img,channels=3)
  except:
    os.remove(TEST_DATASET + image)
print("number of photos in test : " , len(os.listdir(TEST_DATASET)))

TensorFlow 2.x selected.
number of photos in train :  5847
number of photos in test :  831



Now we want to get images and texts with their labels for Humour ,Sarcasm and Offensive

In [0]:
train_images = []
train_texts = []
train_labels = []

max_len_text = 0

for line in train_data : 
  if os.path.exists(TRAIN_DATASET + line[0]):
    train_images.append(TRAIN_DATASET + line[0])
    train_texts.append(line[3])
    if len(line[3]) > max_len_text :
      max_len_text = len(line[3])
    label=[]
    is_other = [1,0]
    if line[4] == 'not_funny':
      label.append([0,1])
    else :
      label.append([1,0])
      is_other = [0,1]
    if line[5] == 'not_sarcastic' or line[5] == 'general':
      label.append([0,1])
    else :
      label.append([1,0])
      is_other = [0,1]
    if line[6] == 'not_offensive' or line[6] == 'slight':
      label.append([0,1])
    else :
      label.append([1,0])
      is_other = [0,1]
     
    label.append(is_other)
    train_labels.append(label)

test_images = []
test_texts = []
test_labels = []

for line in test_data : 
  if os.path.exists(TEST_DATASET + line[0]):
    test_images.append(TEST_DATASET + line[0])
    test_texts.append(line[3])
    if len(line[3]) > max_len_text :
      max_len_text = len(line[3])
    label=[]
    is_other = [1,0]
    if line[4] == 'not_funny':
      label.append([0,1])
    else :
      label.append([1,0])
      is_other = [0,1]
    if line[5] == 'not_sarcastic' or line[5] == 'general':
      label.append([0,1])
    else :
      label.append([1,0])
      is_other = [0,1]
    if line[6] == 'not_offensive' or line[6] == 'slight':
      label.append([0,1])
    else :
      label.append([1,0])
      is_other = [0,1]
    label.append(is_other)
    test_labels.append(label)



**make dataset with tensorflow**

In [0]:
"""libraries we use"""
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import utils
import tensorflow.keras.layers
from tensorflow import keras
from tensorflow.keras import applications
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K

In [0]:
# implementing tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_texts + test_texts)
sequences = tokenizer.texts_to_sequences(train_texts)
train_texts = pad_sequences(sequences, maxlen=max_len_text)
sequences = tokenizer.texts_to_sequences(test_texts)
test_texts = pad_sequences(sequences, maxlen=max_len_text)
word_index = tokenizer.word_index

we need a word embedding , we use glove pre-trained word embedding

In [120]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

embeddings_index = {}
f = open('glove.6B.100d.txt')

for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs

f.close()

print('Found %s word vectors.' % len(embeddings_index))

#  making embedding matrix 
embedding_dim = 100
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector

Found 400000 word vectors.


now we create dataset using tensorflow

In [0]:
def get_images(inputs,outputs) :
  img = tf.io.read_file(inputs[0])
  return (img , inputs[1]) , outputs

def process_image(inputs,outputs):
    img = tf.image.decode_jpeg(inputs[0], channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [200, 200])
    img = img / 255.0
    return (img,inputs[1]),outputs

def decode_label(inputs,outputs):
    return inputs,(outputs[0],outputs[1],outputs[2],outputs[3])


AUTOTUNE = tf.data.experimental.AUTOTUNE
X1 = X2 = X3 = X4 =  AUTOTUNE 
train_dataset = tf.data.Dataset.from_tensor_slices(((train_images,train_texts),train_labels))
train_dataset = train_dataset.shuffle(len(train_images))
train_dataset = train_dataset.map(get_images, num_parallel_calls=X1)
train_dataset = train_dataset.map(process_image,num_parallel_calls=X2)
train_dataset = train_dataset.map(decode_label,num_parallel_calls=X3)
train_dataset = train_dataset.batch(batch_size=30)
train_dataset = train_dataset.prefetch(buffer_size=X4)

AUTOTUNE2 = tf.data.experimental.AUTOTUNE
Y1 = Y2 = Y3 = Y4 =  AUTOTUNE
test_dataset = tf.data.Dataset.from_tensor_slices(((test_images,test_texts),test_labels))
test_dataset = test_dataset.map(get_images, num_parallel_calls=Y1)
test_dataset = test_dataset.map(process_image,num_parallel_calls=Y2)
test_dataset = test_dataset.map(decode_label,num_parallel_calls=Y3)
test_dataset = test_dataset.batch(batch_size=100)
test_dataset = test_dataset.prefetch(buffer_size=Y4)

it is time to create network

In [0]:
def create_model():
  image_input = keras.Input(shape=(200,200,3))
  cnn1 = keras.layers.Conv2D(50,(5,5),activation='relu')(image_input)
  max_pool_1 = keras.layers.MaxPool2D()(cnn1)
  cnn2 = keras.layers.Conv2D(25,(5,5),activation='relu')(max_pool_1)
  max_pool_2 = keras.layers.MaxPool2D()(cnn2)
  cnn3 = keras.layers.Conv2D(25,(4,4),activation='relu')(max_pool_2)
  max_pool_3 = keras.layers.MaxPool2D()(cnn3)
  cnn4 = keras.layers.Conv2D(15,(3,3),activation='relu')(max_pool_3)
  max_pool_4= keras.layers.MaxPool2D()(cnn4)
  cnn5 = keras.layers.Conv2D(5,(3,3),activation='relu')(max_pool_4)
  flatten = keras.layers.Flatten()(cnn5)

  text_input = keras.Input(shape=(max_len_text))
  embedding_layer = keras.layers.Embedding(len(word_index) + 1,embedding_dim,weights=[embedding_matrix],name = 'embedding_layer',trainable=False)
  embedded = embedding_layer(text_input)
  lstm1 = keras.layers.LSTM(300,return_sequences = True)(embedded)
  lstm2 = keras.layers.LSTM(300)(lstm1)

  concatinate = tf.concat([flatten,lstm2] , axis = 1)
  dense1 = keras.layers.Dense(100)(concatinate)
  dense1_0 = keras.layers.Dense(25)(dense1)
  dense2 = keras.layers.Dense(2,activation = 'softmax',name = 'Humour')(dense1_0)
  dense3 = keras.layers.Dense(2,activation = 'softmax',name = 'Sarcasm')(dense1_0)
  dense4 = keras.layers.Dense(2,activation = 'softmax',name = 'Offensive')(dense1_0)
  dense5 = keras.layers.Dense(2,activation = 'softmax',name = 'Other')(dense1_0)

  model = keras.Model([image_input,text_input],[dense2,dense3,dense4,dense5])

  
  tf.keras.utils.plot_model(model, 'model.png', show_shapes=True)
  model.compile(optimizer='adam' , loss='binary_crossentropy',metrics=['acc']) 

  return model

In [135]:
model = create_model()

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint('model_1_B/', monitor='val_loss', verbose=1, save_best_only=True,save_weights_only=True, mode='min')

history = model.fit(train_dataset,epochs = 40,validation_data=test_dataset,callbacks=[checkpoint_callback])

Train for 195 steps, validate for 9 steps
Epoch 1/40
194/195 [============================>.] - ETA: 0s - loss: 2.1519 - Humour_loss: 0.5565 - Sarcasm_loss: 0.5963 - Offensive_loss: 0.5616 - Other_loss: 0.4376 - Humour_acc: 0.7627 - Sarcasm_acc: 0.7220 - Offensive_acc: 0.7586 - Other_acc: 0.8431
Epoch 00001: val_loss improved from inf to 2.12021, saving model to model_1_B/
195/195 [==============================] - 55s 280ms/step - loss: 2.1523 - Humour_loss: 0.5568 - Sarcasm_loss: 0.5961 - Offensive_loss: 0.5616 - Other_loss: 0.4378 - Humour_acc: 0.7625 - Sarcasm_acc: 0.7221 - Offensive_acc: 0.7585 - Other_acc: 0.8430 - val_loss: 2.1202 - val_Humour_loss: 0.5249 - val_Sarcasm_loss: 0.5804 - val_Offensive_loss: 0.5650 - val_Other_loss: 0.4499 - val_Humour_acc: 0.7762 - val_Sarcasm_acc: 0.7316 - val_Offensive_acc: 0.7521 - val_Other_acc: 0.8363
Epoch 2/40
194/195 [============================>.] - ETA: 0s - loss: 2.1227 - Humour_loss: 0.5486 - Sarcasm_loss: 0.5944 - Offensive_loss: 0.55

evaluate the best model 

In [142]:
model = create_model()
model.evaluate(test_dataset)
model.load_weights('model_1_B/')
score = model.evaluate(test_dataset)
avg_acc = (score[-1] + score[-2] + score[-3] + score[-4])/4
print('Total accuracy : ' , avg_acc)

9/9 [==============================] - 5s 553ms/step - loss: 1.9224 - Humour_loss: 0.4702 - Sarcasm_loss: 0.5286 - Offensive_loss: 0.5669 - Other_loss: 0.3567 - Humour_acc: 0.8291 - Sarcasm_acc: 0.7690 - Offensive_acc: 0.7858 - Other_acc: 0.8797
Total accuracy :  0.8158844709396362


we have to get f1-score and recall for this task

In [138]:
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score


Humour_labels = []
Sarcasm_labels = []
Offensive_labels = []
Other_labels = []

score = model.predict(test_dataset)

for _, label in test_dataset: 
    humour_label = label[0]
    sarcasm_label = label[1]
    offensive_label = label[2]
    other_label = label[3]
    for i in humour_label :
      Humour_labels.append(i)
    for i in sarcasm_label :
      Sarcasm_labels.append(i)
    for i in offensive_label :
      Offensive_labels.append(i)
    for i in other_label :
      Other_labels.append(i)

f1_humour = f1_score(Humour_labels,score[0] > 0.5,average='micro')
f1_sarcasm = f1_score(Sarcasm_labels,score[1] > 0.5,average='micro')
f1_offensive = f1_score(Offensive_labels,score[2] > 0.5,average='micro')
f1_other = f1_score(Other_labels,score[3] > 0.5,average='micro')

f1_score = (f1_humour + f1_sarcasm + f1_offensive + f1_other)/4
print('f1_score : ' , f1_score)

recall_humour = recall_score(Humour_labels,score[0] > 0.5,average='micro')
recall_sarcasm = recall_score(Sarcasm_labels,score[1] > 0.5,average='micro')
recall_offensive = recall_score(Offensive_labels,score[2] > 0.5,average='micro')
recall_other = recall_score(Other_labels,score[3] > 0.5,average='micro')

recall = (recall_humour + recall_sarcasm + recall_offensive + recall_other)/4
print('recall : ' , recall)

f1_score :  0.8158844765342962
recall :  0.8158844765342961


In [143]:
!zip -r model_B.zip 'model_1_B'

  adding: model_1_B/ (stored 0%)
  adding: model_1_B/.data-00001-of-00002 (deflated 9%)
  adding: model_1_B/checkpoint (deflated 34%)
  adding: model_1_B/.index (deflated 73%)
  adding: model_1_B/.data-00000-of-00002 (deflated 27%)
